<a href="https://colab.research.google.com/github/mars241/CheckPoints/blob/main/Corr_2023_04_20_Livecoding_Extraction_info_mail_RegEx.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Définition & ressources**

**Une expression régulière est un ensemble de caractères décrivant un motif permettant d'identifier et/ou d'extraire les occurrences d’un terme dans un texte.** Les expressions régulières (motifs) sont utilisées dans la plupart des langages de programmation, mais également par de nombreux outils UNIX (comme grep, sed...) et les fonctions de recherche de certains outils de bureautique. Leur utilisation est universelle, c'est pourquoi il est indispensable de savoir s'en servir.

**[Apprendre pas à pas](https://regexlearn.com/learn/regex101)**


[Cheatsheet en PDF](https://res.cloudinary.com/dyd911kmh/image/upload/v1665049689/Regular_Expressions_Cheat_Sheet_b95aae6488.png)

[Cheatsheet interactive](https://regexlearn.com/cheatsheet)

In [ ]:
import re
import pandas as pd
pd.set_option('display.max_colwidth', None)     # pour afficher tout le contenu des cellules d'un dataframe

# **Exemples divers**

In [ ]:
texte = "Matrix (31/03/1999) - 130 minutes. IMDB note 9/10 (https://www.imdb.com/title/tt0133093). Thomas A. Anderson (Keanu Reeves), un jeune informaticien connu dans le monde du hacking sous le pseudonyme de Neo, est contacté via son ordinateur par ce qu’il pense être un groupe de hackers. Ils lui font découvrir que le monde dans lequel il vit n’est qu’un monde virtuel, la matrice, dans lequel les êtres humains sont gardés sous contrôle. Morpheus (Laurence Fishburne), le capitaine du Nebuchadnezzar, contacte Neo et pense que celui-ci est l’Élu qui peut libérer les êtres humains du joug des machines et prendre le contrôle de la matrice"
texte

'Matrix (31/03/1999) - 130 minutes. IMDB note 9/10 (https://www.imdb.com/title/tt0133093). Thomas A. Anderson (Keanu Reeves), un jeune informaticien connu dans le monde du hacking sous le pseudonyme de Neo, est contacté via son ordinateur par ce qu’il pense être un groupe de hackers. Ils lui font découvrir que le monde dans lequel il vit n’est qu’un monde virtuel, la matrice, dans lequel les êtres humains sont gardés sous contrôle. Morpheus (Laurence Fishburne), le capitaine du Nebuchadnezzar, contacte Neo et pense que celui-ci est l’Élu qui peut libérer les êtres humains du joug des machines et prendre le contrôle de la matrice'

#### **Rechercher toutes les suites de chiffres ('31', '03', '1999', '130', '9', '10', '0133093') contenues dans le texte**

In [ ]:
# \d désigne un chiffre entre 0 et 9, et le caractère + signifie "1 ou plus"

re.findall('\d+', texte)

['31', '03', '1999', '130', '9', '10', '0133093']

In [ ]:
# Écrire juste d (sans l'antislash \) signifie qu'on cherche la lettre d
re.findall('d', texte)    # attrape toutes les lettres d, de façon isolée
# re.findall('d+', texte)   # attrape tous les groupes de lettres d

#### **Rechercher le A. de Thomas A. Anderson**

In [ ]:
re.findall('A\.', texte)

['A.']

#### **Rechercher la date contenue dans le document**

In [ ]:
# re.findall('\d{1,}/\d{1,}/\d{2,4}', texte)      # nombre d'un chiffre minimum/nombre d'un chiffre minimum/nombre entre 2 et 4 chiffres
# re.findall('\d{2}/\d{2}/\d{4}', texte)      # nombre de 2 chiffres/nombre de 2 chiffres/nombre de 4 chiffres
re.findall('\d+/\d+/\d+', texte)

['31/03/1999']

#### **Rechercher la note contenue dans le texte, sans pour autant sélectionner une partie de la date (on peut s'aider du caractère espace avant la note)**

In [ ]:
re.findall('\s(\d+/\d+)', texte)      # attrape ce qui est entre parenthèses seulement
# re.findall('\s\d+/\d+', texte)      # attrape l'espace aussi

['9/10']

#### **Rechercher les mots ayant au moins 14 caractères**

In [ ]:
re.findall('\w{14,}', texte)

['Nebuchadnezzar']

#### **Rechercher l'url de la fiche du film sur IMDB (sans les parenthèses)**

In [ ]:
re.findall(r'\b((?:https?://)\S+)\b', texte)      # merci ChatGPT :D

['https://www.imdb.com/title/tt0133093']

# **Cas pratique : Pandas x RegEx**

#### **Extraire un numéro de téléphone et le stocker dans une nouvelle colonne**

In [ ]:
dftest = pd.DataFrame({"content": ["my number is 0612345678", "hey, my phone number is 0787654321, bye!"]})
dftest

,content
0,my number is 0612345678
1,"hey, my phone number is 0787654321, bye!"


In [ ]:
text = "my number is 0612345678"

In [ ]:
re.findall('\d{10}', text)      # les nombres de 10 chiffres

['0612345678']

In [ ]:
# le résultat est une liste donc [0] pour accéder à la chaîne de caractères dedans
re.findall('\d{10}', text)[0]

'0612345678'

In [ ]:
dftest['phone_number'] = dftest['content'].apply(lambda x : re.findall('\d{10}', x)[0])
dftest

,content,phone_number
0,my number is 0612345678,0612345678
1,"hey, my phone number is 0787654321, bye!",0787654321


#### **Après scraping sur Welcome to the Jungle : extraire nom, prénom, fournisseur d'emails, nom de l'entreprise, titre du job, lieu, et les stocker dans 6 nouvelles colonnes**

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/murpi/wilddata/master/quests/candidates_df.csv')
df

,candidate_email_address,applied_for
0,leo.martin@yandex.com,https://www.welcometothejungle.com/fr/companies/mes-allocs/jobs/bras-droit-ceo_paris
1,gabriel.legrand@gmx.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/sdr-sales-development-representative-stage_paris
2,raphael.bernard@gmx.us,https://www.welcometothejungle.com/fr/companies/work4/jobs/product-management-associate_paris
3,jade.garnier@aol.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/stage-customer-success-manager_paris
4,louise.dubois@icloud.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/inside-sales_paris
...,...,...
263,noham.rodriguez@aol.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/sdr-sales-development-representative_seattle
264,younes.marie@hotmail.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/inside-sales-mid-market_seattle
265,lorenzo.dasilva@outlook.com,https://www.welcometothejungle.com/fr/companies/fabernovel/jobs/project-analyst-in-san-francisco_san-francisco
266,melina.barbier@gmail.com,https://www.welcometothejungle.com/fr/companies/jus-mundi/jobs/general-counsel_paris


###### Colonne candidate_email_address

In [ ]:
# 1. Colonne candidate_email_address
mail = 'leo.martin@yandex.com'

In [ ]:
# Extraire nom, prénom, et fournisseur d'emails en une seule commande
re.findall('\w+', mail)

['leo', 'martin', 'yandex', 'com']

In [ ]:
re.findall('\w+', mail)[0].title()    # la méthode title passe la première lettre en majuscule

'Leo'

###### Colonne applied_for

In [ ]:
# 2. Colonne applied_for
url = 'https://www.welcometothejungle.com/fr/companies/jus mundi/jobs/general-counsel_paris'

In [ ]:
# Nom de l'entreprise = chercher tout ce qui se trouve entre companies/ et /jobs
" ".join(re.findall('companies\/(.*)\/jobs', url)[0].split('-')).title()

'Jus Mundi'

In [ ]:
# Titre du job = chercher tout ce qui se trouve entre jobs/ et _
" ".join(re.findall('jobs/(.*)_', url)[0].split('-')).title()

'General Counsel'

In [ ]:
# Lieu = trouver tout ce qui se trouve après _
re.findall('_(.*)', url)[0].title()

'Paris'

###### Backup

In [ ]:
# Nom entreprise
re.findall('companies\/(.*)\/jobs', url)

# Titre job
re.findall('jobs\/(.*?)\_', url)
re.findall('jobs\/(.*)\_', url)

# Lieu
re.findall('\_(.*)', url)

['jus-mundi']

###### **Apply \o/**

In [ ]:
# On utilise la fonction apply de Pandas et les RegEx définies plus haut pour stocker chaque champ qu'on a extrait dans une nouvelle colonne
df.columns

Index(['candidate_email_address', 'applied_for'], dtype='object')

In [ ]:
df['first_name'] = df['candidate_email_address'].apply(lambda x : re.findall('\w+', x)[0].title())
df

,candidate_email_address,applied_for,first_name
0,leo.martin@yandex.com,https://www.welcometothejungle.com/fr/companies/mes-allocs/jobs/bras-droit-ceo_paris,Leo
1,gabriel.legrand@gmx.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/sdr-sales-development-representative-stage_paris,Gabriel
2,raphael.bernard@gmx.us,https://www.welcometothejungle.com/fr/companies/work4/jobs/product-management-associate_paris,Raphael
3,jade.garnier@aol.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/stage-customer-success-manager_paris,Jade
4,louise.dubois@icloud.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/inside-sales_paris,Louise
...,...,...,...
263,noham.rodriguez@aol.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/sdr-sales-development-representative_seattle,Noham
264,younes.marie@hotmail.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/inside-sales-mid-market_seattle,Younes
265,lorenzo.dasilva@outlook.com,https://www.welcometothejungle.com/fr/companies/fabernovel/jobs/project-analyst-in-san-francisco_san-francisco,Lorenzo
266,melina.barbier@gmail.com,https://www.welcometothejungle.com/fr/companies/jus-mundi/jobs/general-counsel_paris,Melina


In [ ]:
df['last_name'] = df['candidate_email_address'].apply(lambda x : re.findall('\w+', x)[1].title())
df

,candidate_email_address,applied_for,first_name,last_name
0,leo.martin@yandex.com,https://www.welcometothejungle.com/fr/companies/mes-allocs/jobs/bras-droit-ceo_paris,Leo,Martin
1,gabriel.legrand@gmx.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/sdr-sales-development-representative-stage_paris,Gabriel,Legrand
2,raphael.bernard@gmx.us,https://www.welcometothejungle.com/fr/companies/work4/jobs/product-management-associate_paris,Raphael,Bernard
3,jade.garnier@aol.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/stage-customer-success-manager_paris,Jade,Garnier
4,louise.dubois@icloud.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/inside-sales_paris,Louise,Dubois
...,...,...,...,...
263,noham.rodriguez@aol.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/sdr-sales-development-representative_seattle,Noham,Rodriguez
264,younes.marie@hotmail.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/inside-sales-mid-market_seattle,Younes,Marie
265,lorenzo.dasilva@outlook.com,https://www.welcometothejungle.com/fr/companies/fabernovel/jobs/project-analyst-in-san-francisco_san-francisco,Lorenzo,Dasilva
266,melina.barbier@gmail.com,https://www.welcometothejungle.com/fr/companies/jus-mundi/jobs/general-counsel_paris,Melina,Barbier


In [ ]:
df['email_provider'] =  df['candidate_email_address'].apply(lambda x : re.findall('\w+', x)[2].title())
df

,candidate_email_address,applied_for,first_name,last_name,email_provider
0,leo.martin@yandex.com,https://www.welcometothejungle.com/fr/companies/mes-allocs/jobs/bras-droit-ceo_paris,Leo,Martin,Yandex
1,gabriel.legrand@gmx.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/sdr-sales-development-representative-stage_paris,Gabriel,Legrand,Gmx
2,raphael.bernard@gmx.us,https://www.welcometothejungle.com/fr/companies/work4/jobs/product-management-associate_paris,Raphael,Bernard,Gmx
3,jade.garnier@aol.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/stage-customer-success-manager_paris,Jade,Garnier,Aol
4,louise.dubois@icloud.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/inside-sales_paris,Louise,Dubois,Icloud
...,...,...,...,...,...
263,noham.rodriguez@aol.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/sdr-sales-development-representative_seattle,Noham,Rodriguez,Aol
264,younes.marie@hotmail.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/inside-sales-mid-market_seattle,Younes,Marie,Hotmail
265,lorenzo.dasilva@outlook.com,https://www.welcometothejungle.com/fr/companies/fabernovel/jobs/project-analyst-in-san-francisco_san-francisco,Lorenzo,Dasilva,Outlook
266,melina.barbier@gmail.com,https://www.welcometothejungle.com/fr/companies/jus-mundi/jobs/general-counsel_paris,Melina,Barbier,Gmail


In [ ]:
df['company'] = df['applied_for'].apply(lambda x : " ".join(re.findall('companies\/(.*)\/jobs', x)[0].split('-')).title())
df

,candidate_email_address,applied_for,first_name,last_name,email_provider,company
0,leo.martin@yandex.com,https://www.welcometothejungle.com/fr/companies/mes-allocs/jobs/bras-droit-ceo_paris,Leo,Martin,Yandex,Mes Allocs
1,gabriel.legrand@gmx.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/sdr-sales-development-representative-stage_paris,Gabriel,Legrand,Gmx,Work4
2,raphael.bernard@gmx.us,https://www.welcometothejungle.com/fr/companies/work4/jobs/product-management-associate_paris,Raphael,Bernard,Gmx,Work4
3,jade.garnier@aol.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/stage-customer-success-manager_paris,Jade,Garnier,Aol,Work4
4,louise.dubois@icloud.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/inside-sales_paris,Louise,Dubois,Icloud,Work4
...,...,...,...,...,...,...
263,noham.rodriguez@aol.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/sdr-sales-development-representative_seattle,Noham,Rodriguez,Aol,Letsignit
264,younes.marie@hotmail.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/inside-sales-mid-market_seattle,Younes,Marie,Hotmail,Letsignit
265,lorenzo.dasilva@outlook.com,https://www.welcometothejungle.com/fr/companies/fabernovel/jobs/project-analyst-in-san-francisco_san-francisco,Lorenzo,Dasilva,Outlook,Fabernovel
266,melina.barbier@gmail.com,https://www.welcometothejungle.com/fr/companies/jus-mundi/jobs/general-counsel_paris,Melina,Barbier,Gmail,Jus Mundi


In [ ]:
df['job_title'] = df['applied_for'].apply(lambda x : " ".join(re.findall('jobs/(.*)_', x)[0].split('-')).title())
df

,candidate_email_address,applied_for,first_name,last_name,email_provider,company,job_title
0,leo.martin@yandex.com,https://www.welcometothejungle.com/fr/companies/mes-allocs/jobs/bras-droit-ceo_paris,Leo,Martin,Yandex,Mes Allocs,Bras Droit Ceo
1,gabriel.legrand@gmx.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/sdr-sales-development-representative-stage_paris,Gabriel,Legrand,Gmx,Work4,Sdr Sales Development Representative Stage
2,raphael.bernard@gmx.us,https://www.welcometothejungle.com/fr/companies/work4/jobs/product-management-associate_paris,Raphael,Bernard,Gmx,Work4,Product Management Associate
3,jade.garnier@aol.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/stage-customer-success-manager_paris,Jade,Garnier,Aol,Work4,Stage Customer Success Manager
4,louise.dubois@icloud.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/inside-sales_paris,Louise,Dubois,Icloud,Work4,Inside Sales
...,...,...,...,...,...,...,...
263,noham.rodriguez@aol.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/sdr-sales-development-representative_seattle,Noham,Rodriguez,Aol,Letsignit,Sdr Sales Development Representative
264,younes.marie@hotmail.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/inside-sales-mid-market_seattle,Younes,Marie,Hotmail,Letsignit,Inside Sales Mid Market
265,lorenzo.dasilva@outlook.com,https://www.welcometothejungle.com/fr/companies/fabernovel/jobs/project-analyst-in-san-francisco_san-francisco,Lorenzo,Dasilva,Outlook,Fabernovel,Project Analyst In San Francisco
266,melina.barbier@gmail.com,https://www.welcometothejungle.com/fr/companies/jus-mundi/jobs/general-counsel_paris,Melina,Barbier,Gmail,Jus Mundi,General Counsel


In [ ]:
df['city'] = df['applied_for'].apply(lambda x : re.findall('_(.*)', x)[0].capitalize())
df

,candidate_email_address,applied_for,first_name,last_name,email_provider,company,job_title,city
0,leo.martin@yandex.com,https://www.welcometothejungle.com/fr/companies/mes-allocs/jobs/bras-droit-ceo_paris,Leo,Martin,Yandex,Mes Allocs,Bras Droit Ceo,Paris
1,gabriel.legrand@gmx.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/sdr-sales-development-representative-stage_paris,Gabriel,Legrand,Gmx,Work4,Sdr Sales Development Representative Stage,Paris
2,raphael.bernard@gmx.us,https://www.welcometothejungle.com/fr/companies/work4/jobs/product-management-associate_paris,Raphael,Bernard,Gmx,Work4,Product Management Associate,Paris
3,jade.garnier@aol.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/stage-customer-success-manager_paris,Jade,Garnier,Aol,Work4,Stage Customer Success Manager,Paris
4,louise.dubois@icloud.com,https://www.welcometothejungle.com/fr/companies/work4/jobs/inside-sales_paris,Louise,Dubois,Icloud,Work4,Inside Sales,Paris
...,...,...,...,...,...,...,...,...
263,noham.rodriguez@aol.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/sdr-sales-development-representative_seattle,Noham,Rodriguez,Aol,Letsignit,Sdr Sales Development Representative,Seattle
264,younes.marie@hotmail.com,https://www.welcometothejungle.com/fr/companies/letsignit/jobs/inside-sales-mid-market_seattle,Younes,Marie,Hotmail,Letsignit,Inside Sales Mid Market,Seattle
265,lorenzo.dasilva@outlook.com,https://www.welcometothejungle.com/fr/companies/fabernovel/jobs/project-analyst-in-san-francisco_san-francisco,Lorenzo,Dasilva,Outlook,Fabernovel,Project Analyst In San Francisco,San-francisco
266,melina.barbier@gmail.com,https://www.welcometothejungle.com/fr/companies/jus-mundi/jobs/general-counsel_paris,Melina,Barbier,Gmail,Jus Mundi,General Counsel,Paris


###### Solution proposée de la quête sur Odyssey : extraire 3 champs en 2 lignes

In [ ]:
unpack_email = lambda x: pd.Series([i.title() for i in re.findall(r'([^@.]+)', x['candidate_email_address'])][:-1])
df[['firstname', 'lastname', 'email_service']] = df.apply(unpack_email, axis=1)
df